This notebook provides a representative example of the fine-tuning process. The fundamental script and methodology were applied consistently across all other experiments, with adjustments made to the hyperparameters for each run.

*   Model Quantization
*   LoRA Configuration
*   Training
*   Experiment Tracking




# Finetuning Gemma 3 12B IT Model

In [ ]:
import os
import math
import torch
import wandb
import shutil
from datasets import load_from_disk
from google.colab import drive
from huggingface_hub import login
from transformers import (
    TrainingArguments,
    Trainer,
    AutoModelForCausalLM,
    AutoTokenizer,
    EarlyStoppingCallback,
    BitsAndBytesConfig
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [ ]:
# Google Drive and Paths

drive.mount('/content/drive')
model_path = "/content/drive/MyDrive/ClassicalLatinPoetryGeneration/FinalModels/gemma-3-12b-it"
dataset_drive_path = r"/content/drive/MyDrive/ClassicalLatinPoetryGeneration/DatasetV4-IT/DatasetV4/tokenized"
output_dir = r"/content/drive/MyDrive/ClassicalLatinPoetryGeneration/models/gemma-3-12b-it-FT14"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Wandb
wandb.login()

In [ ]:
# Quantizing the Model, Loading Tokenizer
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)

tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Configuring Lora Adapters
lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# Checking out trainable parameters
model.print_trainable_parameters()

In [ ]:
# Loading the dataset

local_dataset_path = "/content/dataset_local_12b_v4"
if os.path.exists(local_dataset_path):
    shutil.rmtree(local_dataset_path)
shutil.copytree(dataset_drive_path, local_dataset_path)

tokenized_dataset = load_from_disk(local_dataset_path)

In [ ]:
# Defining W&B run configuration
run_config = {
    "learning_rate": 2e-4,
    "lr_scheduler_type": "cosine",
    "epochs": 20,
    "r": lora_config.r,
    "lora_alpha": lora_config.lora_alpha,
    "early_stopping_patience": 3,
    "dataset_version": "V4-gemma-12b-it",
    "batch_size": 4,
    "gradient_accumulation": 4,
    "double_quant": False
}

# New name for the Run:
run_name = f"gemma-3-12b-it_FT14_DSv4_LR{run_config['learning_rate']}_R{run_config['r']}_B{run_config['batch_size']}GA{run_config['gradient_accumulation']}_MOREMODULE_MOREBATCH"
wandb.init(
    project="latin-hexameter-thesis",
    name=run_name,
    config=run_config
)

In [ ]:
# Custom Trainer class to log perplexity
class MyTrainer(Trainer):
    def evaluate(self, *args, **kwargs):
        metrics = super().evaluate(*args, **kwargs)
        try:
            perplexity = math.exp(metrics["eval_loss"])
            custom_metrics = {"eval_perplexity": perplexity}
            self.log(custom_metrics)
        return metrics

# Training arguments for QLoRA
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=run_config["epochs"],
    learning_rate=run_config["learning_rate"],
    lr_scheduler_type=run_config["lr_scheduler_type"],
    per_device_train_batch_size=run_config["batch_size"],
    gradient_accumulation_steps=run_config["gradient_accumulation"],
    bf16=True,
    eval_strategy="steps",
    eval_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=4,
    logging_strategy="steps",
    logging_steps=10,
    report_to="wandb",
)

# Initializing
trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=run_config["early_stopping_patience"])]
)

In [ ]:
trainer.train()

In [ ]:
wandb.finish()